In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf

In [ ]:
df = pd.read_csv("RRCA_baseflow.csv")
# display(df)
df['Date'] = df['Date'] - 693963
df['Date_YMD'] = pd.to_datetime(df['Date'], origin='1900-01-01', unit='D')
df['Month'] = df['Date_YMD'].dt.month
df = df[df['Observed'] != 0]
df = df.dropna()
display(df)
# df.to_csv('RRCA_baseflow_YMD.csv')

# df_train = df[(df['Date_YMD'] >= '1941-01-01') & (df['Date_YMD'] <= '1989-12-31')]
# df_test = df[df['Date_YMD'] >= '1990-01-01']

# Display the first few rows of each dataframe
# print("Data from 1941 to 1989")
# display(df_train.head())

# print("\nData from 1990 to most recent")
# display(df_test.head())

In [ ]:
print(df.isnull().sum())

In [ ]:
unique_segment_ids = df['Segment_id'].unique()
print(unique_segment_ids)

In [ ]:
df.shape

#### Trying KMeans like they do in the paper

In [ ]:
predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']
target = 'Observed'
scaler = StandardScaler()
scaled_vars = scaler.fit_transform(df[predictors])

kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(scaled_vars)

for cluster_id in df['Cluster'].unique():
    df_cluster = df[df['Cluster'] == cluster_id]
    X_cluster = df_cluster[predictors]
    y_cluster = df_cluster[target]

    X_train, X_test, y_train, y_test = train_test_split(X_cluster, y_cluster, test_size=0.2, random_state=42)
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    r2_cluster = r2_score(y_test, lm.predict(X_test))

    print(f"Cluster {cluster_id} R-squared: {r2_cluster:.4f}")

In [ ]:
plt.figure(figsize=(16, 12))

predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']
for i, col in enumerate(predictors, 1):
    plt.subplot(2, 3, i)
    sns.scatterplot(data=df, x=col, y='Observed')
    plt.title(f'{col} vs Observed Baseflow')
plt.tight_layout()
plt.show()

predictors_with_observed = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping', 'Observed']
plt.figure(figsize=(16, 10))
for i, col in enumerate(predictors_with_observed, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

plt.figure(figsize=(30, 8))
for i, col in enumerate(predictors_with_observed, 1):
    plt.subplot(1, 4, i)
    sns.lineplot(data=df, x='Month', y=col, estimator='mean', errorbar=None, color='blue')
    plt.title(f'Monthly Average of {col}')
    plt.xlabel('Month')
    plt.ylabel(col)
    plt.grid(True)
plt.tight_layout()
plt.show()

In [7]:
# specific_segment_id = 144 
# df_specific_segment = df[df['Segment_id'] == specific_segment_id]
# print(f"Data for Segment id {specific_segment_id}:")
# display(df_specific_segment.head())
# # df_specific_segment.to_csv(f'{specific_segment_id}segment.csv')

# plt.figure(figsize=(16, 12))
# predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']

# for i, col in enumerate(predictors, 1):
#     plt.subplot(2, 3, i)
#     sns.scatterplot(data=df_specific_segment, x=col, y='Observed')
#     plt.title(f'{col} vs Observed Baseflow')
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(30, 8))
# predictors_with_observed = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping','Observed']
# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(1, 4, i)
#     sns.scatterplot(data=df_specific_segment, x='Date_YMD', y=col, label=col, color='blue', alpha=0.6)
#     plt.title(f'{col} Over Time')
#     plt.xlabel('Date')
#     plt.ylabel(col)
#     plt.xticks(rotation=45)
#     plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(16, 10))
# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(2, 3, i)
#     sns.histplot(df_specific_segment[col], kde=True)
#     plt.title(f'Distribution of {col}')

# plt.tight_layout()
# plt.show()

In [8]:
specific_segment_id = 171 
df_specific_segment = df[df['Segment_id'] == specific_segment_id]
print(f"Data for Segment id {specific_segment_id}:")
display(df_specific_segment.head())
# df_specific_segment.to_csv(f'{specific_segment_id}segment.csv')

predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']
target = 'Observed'

# plt.figure(figsize=(16, 12))
# for i, col in enumerate(predictors, 1):
#     plt.subplot(2, 3, i)
#     sns.scatterplot(data=df_specific_segment, x=col, y='Observed')
#     plt.title(f'{col} vs Observed Baseflow')
# plt.tight_layout()
# plt.show()

plt.figure(figsize=(30, 8))
predictors_with_observed = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping','Observed']

# for i, col in enumerate(predictors_with_observed, 1):
#     plt.subplot(1, 4, i)
#     sns.scatterplot(data=df_specific_segment, x='Date_YMD', y=col, label=col, color='blue', alpha=0.6)
#     plt.title(f'{col} Over Time')
#     plt.xlabel('Date')
#     plt.ylabel(col)
#     plt.xticks(rotation=45)
#     plt.grid(True)
# plt.tight_layout()
# plt.show()

plt.figure(figsize=(16, 10))
for i, col in enumerate(predictors_with_observed, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df_specific_segment[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

plt.figure(figsize=(30, 8))
for i, col in enumerate(predictors_with_observed, 1):
    plt.subplot(1, 4, i)
    sns.lineplot(data=df_specific_segment, x='Month', y=col, estimator='mean', errorbar=None, color='blue')
    plt.title(f'Monthly Average of {col}')
    plt.xlabel('Month')
    plt.ylabel(col)
    plt.grid(True)
plt.tight_layout()
plt.show()

x = df_specific_segment[predictors].values
y = df_specific_segment[target].values

lm = LinearRegression()
lm.fit(x, y)
print(lm.intercept_)
print(lm.coef_)

formula = 'Observed ~ Evapotranspiration + Precipitation + Irrigation_pumping'
lms = smf.ols(formula=formula, data=df_specific_segment).fit()

print("Coefficients")
display(lms.params)

print("Confidence Intervals")
display(lms.conf_int())

print("Model Summary")
print(lms.summary())

print("P-values")
display(lms.pvalues)


#### This looks at data and distribution over time per segment

In [ ]:
for segment in unique_segment_ids:
    specific_segment_id = segment
    df_specific_segment = df[df['Segment_id'] == specific_segment_id]
    print(f"Data for Segment id {specific_segment_id}:")
    # display(df_specific_segment.head())
    # df_specific_segment.to_csv(f'{specific_segment_id}segment.csv')
    predictors = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']
    target = ['Observed']

    # plt.figure(figsize=(16, 12))
    # for i, col in enumerate(predictors, 1):
    #     plt.subplot(2, 3, i)
    #     sns.scatterplot(data=df_specific_segment, x=col, y='Observed')
    #     plt.title(f'{col} vs Observed Baseflow')
    # plt.tight_layout()
    # plt.show()

    plt.figure(figsize=(30, 8))
    predictors_with_observed = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping','Observed']

    # for i, col in enumerate(predictors_with_observed, 1):
    #     plt.subplot(1, 4, i)
    #     sns.scatterplot(data=df_specific_segment, x='Date_YMD', y=col, label=col, color='blue', alpha=0.6)
    #     plt.title(f'{col} Over Time')
    #     plt.xlabel('Date')
    #     plt.ylabel(col)
    #     plt.xticks(rotation=45)
    #     plt.grid(True)
    # plt.tight_layout()
    # plt.show()

    # plt.figure(figsize=(16, 10))
    # for i, col in enumerate(predictors_with_observed, 1):
    #     plt.subplot(2, 3, i)
    #     sns.histplot(df_specific_segment[col], kde=True)
    #     plt.title(f'Distribution of {col}')
    # plt.tight_layout()
    # plt.show()

    # plt.figure(figsize=(30, 8))
    # for i, col in enumerate(predictors_with_observed, 1):
    #     plt.subplot(1, 4, i)
    #     sns.lineplot(data=df_specific_segment, x='Month', y=col, estimator='mean', color='blue')
    #     plt.title(f'Monthly Average of {col}')
    #     plt.xlabel('Month')
    #     plt.ylabel(col)
    #     plt.grid(True)
    # plt.tight_layout()
    # plt.show()
    
    # plt.figure(figsize=(30, 8))
    # for i, col in enumerate(predictors_with_observed, 1):
    #     plt.subplot(1, 4, i)
    #     sns.scatterplot(data=df_specific_segment, x='Month', y=col, color='blue', alpha=0.6)
    #     plt.title(f'{col} by Month')
    #     plt.xlabel('Month')
    #     plt.ylabel(col)
    #     plt.grid(True)
    # plt.tight_layout()
    # plt.show()

    x = df_specific_segment[predictors].values
    y = df_specific_segment[target].values.ravel()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    print("____Sklearn____")
    lm = LinearRegression()
    lm.fit(x_train, y_train)
    print("Intercept")
    print(lm.intercept_)
    print("Coefficents")
    print(lm.coef_)
    print("Predictions on test sample")
    display(lm.predict(x_test))

    print("____Statsmodel____")
    formula = 'Observed ~ Evapotranspiration + Precipitation + Irrigation_pumping'
    lms = smf.ols(formula=formula, data=df_specific_segment).fit()
    print("Coefficients")
    display(lms.params)
    print("Confidence Intervals")
    display(lms.conf_int())
    print("Model Summary")
    print(lms.summary())
    print("P-values")
    display(lms.pvalues)